<a href="https://colab.research.google.com/github/nweissmueller/COLAB/blob/main/Public_Search_WebMD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install langchain tiktoken duckduckgo-search
!pip install openai
!pip show langchain

In [ ]:
## load OpenAI API key from file on Google Drive
!wget -O api_key.txt 'https://drive.google.com/uc?export=download&id=FILE_ID'

# Open and read the downloaded file
with open('api_key.txt', 'r') as f:
    api_key = f.read().strip()

import os
os.environ["OPENAI_API_KEY"] = api_key

--2023-07-18 00:20:54--  https://drive.google.com/uc?export=download&id=1OkncvrPTnTWlrWVLQksu9Beu1K3ooqhi
Resolving drive.google.com (drive.google.com)... 209.85.145.102, 209.85.145.100, 209.85.145.139, ...
Connecting to drive.google.com (drive.google.com)|209.85.145.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-00-58-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/5nn5g8uurrgi6s93pmc14cg8q1g8ngvk/1689639600000/06267039865044807031/*/1OkncvrPTnTWlrWVLQksu9Beu1K3ooqhi?e=download&uuid=b7189c1e-9e5e-44fd-8bec-6c3f4947fe8d [following]
--2023-07-18 00:20:55--  https://doc-00-58-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/5nn5g8uurrgi6s93pmc14cg8q1g8ngvk/1689639600000/06267039865044807031/*/1OkncvrPTnTWlrWVLQksu9Beu1K3ooqhi?e=download&uuid=b7189c1e-9e5e-44fd-8bec-6c3f4947fe8d
Resolving doc-00-58-docs.googleusercontent.com (doc-00-58-docs.googleusercontent.com)... 142.250.136.132, 2607

## Custom Agents with Custom Search

Source: https://www.youtube.com/watch?v=6UFtRwWnHws




In [ ]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate

from langchain import OpenAI, LLMChain
from langchain.tools import DuckDuckGoSearchRun

from typing import List, Union
from langchain.schema import AgentAction, AgentFinish
import re
import langchain

## Setup Tools

In [ ]:
def duck_wrapper(input_text):
    search_results = search.run(f"site:webmd.com {input_text}")
    return search_results

search = DuckDuckGoSearchRun()

tools = [
    Tool(
        name = "Search WebMD",
        func=duck_wrapper,
        description="useful for when you need to answer medical and pharmalogical questions"
    )
]

## Prompt Template
This instructs the agent on what to do. Generally, the template should incorporate:

**`tools:`** which tools the agent has access and how and when to call them.

**`intermediate_steps:`** These are tuples of previous (**`AgentAction`**, **`Observation`**) pairs. These are generally not passed directly to the model, but the prompt template formats them in a specific way.

**`input:`** generic user input

In [ ]:
# Set up a prompt template
class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]

    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)

## Custom Output Parser
The output parser is responsible for parsing the LLM output into AgentAction and AgentFinish. This usually depends heavily on the prompt used.

This is where you can change the parsing to do retries, handle whitespace, etc

In [ ]:
class CustomOutputParser(AgentOutputParser):

    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

In [ ]:
output_parser = CustomOutputParser()

## Set up LLM
Choose the LLM you want to use!

In [ ]:
llm = OpenAI(temperature=0)

## Define the stop sequence
This is important because it tells the LLM when to stop generation.

This depends heavily on the prompt and model you are using. Generally, you want this to be whatever token you use in the prompt to denote the start of an Observation (otherwise, the LLM may hallucinate an observation for you).

## Adding a Conversation Memory
If you want to add memory to the agent, you’ll need to:

1. Add  **`chat_history`** into the custom prompt

2. Add the memory object to the agent executor when defining it.

## Set up the Agent
We can now combine everything to set up our agent


### What is an Agent Executor?

Agent Executors take an agent and tools and use the agent to decide which tools to call and in what order.



In [ ]:
# Set up the base template
template_with_history = """Answer the following questions as best you can, but speaking as compasionate medical professional. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to speak as a compasionate medical professional when giving your final answer. If the condition is serious advise they speak to a doctor.

Previous conversation history:
{history}

New question: {input}
{agent_scratchpad}"""

In [ ]:
prompt_with_history = CustomPromptTemplate(
    template=template_with_history,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps", "history"]
)

In [ ]:
llm_chain = LLMChain(llm=llm, prompt=prompt_with_history)

In [ ]:
tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=tool_names
)

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

memory=ConversationBufferWindowMemory(k=2)

In [ ]:
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
    memory=memory
    )

In [ ]:
agent_executor.run("How can I treat a sprained adductor?")



> Entering new AgentExecutor chain...

Thought: I need to find out what the best treatment for a sprained adductor is.
Action: Search WebMD
Action Input: "Sprained adductor treatment"

Observation:There are two main causes of hip impingement: A deformity of the ball at the top of the femur (called cam impingement). If the head is not shaped normally, the abnormal part of the head can jam in ... Being overweight. Stress fractures in your toes or metatarsals. Bunions. Torn ligaments in your feet. A condition known as Morton's neuroma (interdigital neuroma) also causes metatarsalgia-like ... 5. Hot and cold may be opposites, but both can help keep you comfortable. Cold treatment is usually best for an injury that just happened. After about 72 hours, doctors usually suggest switching ... Excessive activity, like too much running, can cause bleeding or swelling inside them. If you have chronic compartment syndrome, a pressure buildup inside your muscles causes your blood flow to go ... Na

'The best treatment for a sprained adductor is to apply cold treatment for the first 72 hours, followed by hot and cold treatments to reduce pain and swelling. If the pain persists, it is important to speak to a doctor.'

In [ ]:
agent_executor.run("What meds could I take?")



> Entering new AgentExecutor chain...
Thought: I need to find out what medications are available for this condition.
Action: Search WebMD
Action Input: Sprained adductor medications

Observation:There are two main causes of hip impingement: A deformity of the ball at the top of the femur (called cam impingement). If the head is not shaped normally, the abnormal part of the head can jam in ... National Collaborating Centre for Women's and Children's Health: "Monitoring and Pain Relief for Induction of Labor." National Institutes of Health: "Induced labor at 39 weeks may reduce ... Pulled to one direction. Other symptoms that may accompany vertigo include: Feeling nauseated. Vomiting. Abnormal or jerking eye movements (nystagmus) Headache. Sweating. Ringing in the ears or ... Xylazine, commonly known as "tranq," is a non-opioid sedative analgesic medication that's largely mixed into (adulterated) and used as an additive with other opioid substances like heroin ... Take note of your med

'There are several medications available to treat a sprained adductor, including non-opioid sedative analgesics, such as Xylazine, and other medications that can help reduce pain and swelling. It is important to speak to a doctor before taking any medications, as they can advise on the best course of treatment for your individual situation.'

In [ ]:
agent_executor.run("How long will it take to heal?")